### Урок 14. Transfer learning

  1. взять данные из
    https://www.kaggle.com/datasets/mrapplexz/bashim-quotes \
    обучить модель GPT для генерации своих цитат

  2. взять новостные данные из
    https://github.com/natasha/corus \
    load_lenta2 \
    нам понадобиться сам текст и заголовок \
    обучить модель T5/ или GPT для генерации заголовков для статей


!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2

In [3]:
!pip install transformers corus sentencepiece -q

#### 1

In [4]:
import pandas as pd
import json
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-08 12:38:26.101376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


!unzip "data/archive.zip"

In [6]:
DATASET_PATH = 'dataset.jsonl'

In [8]:
with open(DATASET_PATH) as f:
     df = pd.read_json(DATASET_PATH, lines=True).set_index('id')
df.head()

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...


In [9]:
df.drop(['date', 'rating'], axis=1, inplace=True)
df.head()

,text
id,
1,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,<томатик_рад> а ты не чувствуешь красоту мира?...
3,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,<Ohtori_Akio> мы - как разработчики - живём с ...


In [10]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Downloading: 100%|██████████████████████████████| 608/608 [00:00<00:00, 136kB/s]
Downloading: 100%|█████████████████████████| 1.71M/1.71M [00:00<00:00, 2.06MB/s]
Downloading: 100%|█████████████████████████| 1.27M/1.27M [00:00<00:00, 1.54MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Downloading: 100%|███████████████████████████| 551M/551M [00:48<00:00, 11.4MB/s]


In [11]:
sep = '\n***\n'
prefix = sep.join([''] + random.sample(list(df['text']), k=5) + [''])
tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]
print(prefix)


***
xxx: Чо как?
yyy: Да вот, на работе закрыли доступ к вк, сижу теперь на hh
***
<Пинг> бля, мне даже в sims2 не дают!
***
xxx: В армии ж как: круглое - несут, а квадратное - катят. 
xxx: Вот стою и смотрю на снеговика в нашей части. Квадратный. 
xxx: Все правильно - накатали.
***
< mary.j > одела я сегодня свою чёрную кофточку с декольте, ну и иду платить за кредит. Иду вниз на саксаганского стоят 3 парня и лупятся на мои сиськт :"вот это да", "круто"... Иду наверх они же и то же самое. Я "мальчики, вы что сисек не видили?" . Один из них : "да обычно либо сиськи либо лицо. А Вам так к лицу Ваши сиськи".
***
- аватар у тебя клевый :)
- ага!!! а что это?
***



In [12]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    do_sample=False, 
    max_length=size+50, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df.loc[:10000, 'text'], test_size=0.15)

In [14]:
import re

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\[\w+\]", "", summary)
        summary = re.sub(r"<[\w+,\!, -]>", "", summary)
        summary = re.sub(r"<\w+>", "", summary)
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)
  
build_text_files(train,'./train_dataset.txt')
build_text_files(test,'./test_dataset.txt')

In [15]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 1666
Test dataset length: 294


In [16]:
train_path = './train_dataset.txt'
test_path = './test_dataset.txt'

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [17]:
training_args = TrainingArguments(
    output_dir="./GPT/gpt2-train", 
    overwrite_output_dir=True, 
    num_train_epochs=3, 
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,  
    eval_steps = 400, 
    save_steps=800, 
    warmup_steps=500,
    )

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [19]:
trainer.train()

/usr/local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 676
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 507
  Number of trainable parameters = 125231616


Step,Training Loss
500,4.273200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=507, training_loss=4.267856063692292, metrics={'train_runtime': 3227.2923, 'train_samples_per_second': 0.628, 'train_steps_per_second': 0.157, 'total_flos': 132475060224000.0, 'train_loss': 4.267856063692292, 'epoch': 3.0})

In [20]:
trainer.save_model()
tokenizer.save_pretrained('GPT/gpt2-train')
model.save_pretrained('GPT/model_gpt2')

Saving model checkpoint to ./GPT/gpt2-train
Configuration saved in ./GPT/gpt2-train/config.json
Model weights saved in ./GPT/gpt2-train/pytorch_model.bin
tokenizer config file saved in GPT/gpt2-train/tokenizer_config.json
Special tokens file saved in GPT/gpt2-train/special_tokens_map.json
Configuration saved in GPT/model_gpt2/config.json
Model weights saved in GPT/model_gpt2/pytorch_model.bin


In [21]:
tokenizer = AutoTokenizer.from_pretrained("GPT/gpt2-train")
model_new = AutoModelForCausalLM.from_pretrained("GPT/model_gpt2")

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file GPT/model_gpt2/config.json
Model config GPT2Config {
  "_name_or_path": "GPT/model_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dty

In [22]:
size = tokens['input_ids'].shape[1]
output = model_new.generate(
    **tokens, 
    do_sample=False, 
    max_length=size+100, 
    repetition_penalty=5., 
    temperature=0.5,
    num_beams=10,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ia http://www.livejournal.com/users/maxim_korolev/126565.html#cutid1  <@Luftwaffe> У меня есть подозрение, что кто-то очень хочет со мной поговорить по душам... (с) * Viktor has quit IRC (Viktor is now known as viktor_officer) [Quit]


#### 2

In [23]:
from corus import load_lenta2

path = 'data/lenta-ru-news.csv.bz2'
records = load_lenta2(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/hungarnn/',
    title='1914. Русские войска вступили в\xa0пределы Венгрии  ',
    text='Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии

In [24]:
data = [(record.title, record.text) for record in records]
df_news = pd.DataFrame({'title': [record[0] for record in data], 'text': [record[1] for record in data]})
df_news.head()

,title,text
0,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про..."
1,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
2,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...
3,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю..."
4,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...


In [25]:
df_news.shape

(800974, 2)

In [26]:
df_train, df_test = train_test_split(df_news[:2000], test_size=0.2)

In [27]:
from datasets import Dataset

df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text'],
     num_rows: 1600
 }),
 Dataset({
     features: ['title', 'text'],
     num_rows: 400
 }))

In [36]:
dataset_test['text'][0]

'Министр внутренних дел России Владимир Рушайло считает, что чеченский муфтий Ахмед Кадыров может стать "альтернативной фигурой" при переговорах между федеральным центром и Чечней по урегулированию ситуации в республике. Об этом он сообщил в четверг в беседе с журналистами, передает ИТАР-ТАСС. Глава МВД напомнил, что председатель правительства Владимир Путин и муфтий Кадыров встречались накануне и провели успешный диалог. Рушайло с удовлетворением отметил, что "политический диалог уже начался", однако он подчеркнул, что "мы пока не делаем далеко идущих выводов". Рушайло в очередной раз подчеркнул, что Аслан Масхадов не может представлять чеченскую сторону на переговорах. Напомним, что 11 октября Масхадов заявил, что он освобождает от должности муфтия чеченской республики Ахмеда-Хаджи Кадырова. Масхадов объяснил тогда свое решение (весьма сомнительное с точки зрения шариата) тем, что муфтий "пытается развязать гражданскую войну в Чечне".'

In [38]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_len_text = max(map(lambda txt: len(txt.split()), dataset_train['text']))
max_len_tlt = max(map(lambda txt: len(txt.split()), dataset_train['title']))

def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_len_txt)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tlt)
    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

loading file spiece.model from cache at /Users/dv/.cache/huggingface/hub/models--IlyaGusev--rut5_base_sum_gazeta/snapshots/f09a08cae5d74c70e55da1a6ebb49f88c26f433b/spiece.model
loading file tokenizer.json from cache at /Users/dv/.cache/huggingface/hub/models--IlyaGusev--rut5_base_sum_gazeta/snapshots/f09a08cae5d74c70e55da1a6ebb49f88c26f433b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /Users/dv/.cache/huggingface/hub/models--IlyaGusev--rut5_base_sum_gazeta/snapshots/f09a08cae5d74c70e55da1a6ebb49f88c26f433b/special_tokens_map.json
loading file tokenizer_config.json from cache at /Users/dv/.cache/huggingface/hub/models--IlyaGusev--rut5_base_sum_gazeta/snapshots/f09a08cae5d74c70e55da1a6ebb49f88c26f433b/tokenizer_config.json
100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 143.74ba/s]


In [39]:
dataset_train.save_to_disk('lenta2/train')
dataset_test.save_to_disk('lenta2/test')

In [40]:
from transformers import T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading: 100%|██████████████████████████████| 766/766 [00:00<00:00, 439kB/s]
loading configuration file config.json from cache at /Users/dv/.cache/huggingface/hub/models--IlyaGusev--rut5_base_sum_gazeta/snapshots/f09a08cae5d74c70e55da1a6ebb49f88c26f433b/config.json
Model config T5Config {
  "_name_or_path": "cointegrated/rut5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 2,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 200,
  "model_type": "t5",
  "num_beams": 5,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings"

In [41]:
output_dir = 'lenta2/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=1000,
    remove_unused_columns=True,
    eval_steps=500, 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
from datasets import load_from_disk

dataset_train = load_from_disk("lenta2/train")
dataset_test = load_from_disk("lenta2/test")
dataset_train, dataset_test

(Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1600
 }),
 Dataset({
     features: ['title', 'text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 400
 }))

In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: title, text. If title, text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 2400
  Number of trainable parameters = 244309248


Step,Training Loss
500,3.049400
1000,2.126100
1500,1.746900
2000,1.497400


Saving model checkpoint to lenta2/output/checkpoint-1000
Configuration saved in lenta2/output/checkpoint-1000/config.json
Model weights saved in lenta2/output/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to lenta2/output/checkpoint-2000
Configuration saved in lenta2/output/checkpoint-2000/config.json
Model weights saved in lenta2/output/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2400, training_loss=1.9987492879231772, metrics={'train_runtime': 12905.7855, 'train_samples_per_second': 0.372, 'train_steps_per_second': 0.186, 'total_flos': 2549012889600000.0, 'train_loss': 1.9987492879231772, 'epoch': 3.0})

In [44]:
trainer.save_model(output_dir + '/model')

Saving model checkpoint to lenta2/output/model
Configuration saved in lenta2/output/model/config.json
Model weights saved in lenta2/output/model/pytorch_model.bin


In [54]:
def title_pred(idx):
    input_text = dataset_test['text'][idx]
    input_title = dataset_test['title'][idx]

    use_cuda = False
    device = torch.device("cpu")

    with torch.no_grad():
        tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt').to(device)
        source_ids = tokenized_text['input_ids'].to(dtype = torch.long)
        source_mask = tokenized_text['attention_mask'].to(dtype = torch.long)
        generated_ids = model.generate(
            input_ids = source_ids,
            attention_mask = source_mask, 
            max_length=1512,
            num_beams=7,
            temperature = 1.3,
            repetition_penalty=1, 
            length_penalty=1, 
            early_stopping=True,
            no_repeat_ngram_size=2
            ).to(device)
        pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    print("Text:\n" + input_text)
    print("Real title: " + input_title)
    print("Pred title: " + pred)

In [55]:
title_pred(1)

Text:
Мэру Москвы Юрию Лужкову вероятность отставки президента и проведения в России досрочных президентских выборов кажется достаточно реальной. Как сообщает ИТАР-ТАСС, мэр заявил, что глава российского государства не может "по состоянию здоровья эффективно выполнять свои обязанности". В принципе, считает Лужков, вариант проведения досрочных президентских выборов вполне приемлем. Однако их совмещение по срокам с выборами в Государственную Думу недопустимо, поскольку избирателей "нельзя лишать права сделать выбор из более широкого круга претендентов". Вывод из игры "неугодных кандидатов" невозможен, отметил Юрий Лужков. Наоборот, в выборах главы государства необходимо участие всех претендентов. "В противном случае народ не будет иметь возможности сделать полноразмерный выбор", - добавил он. Мэр Москвы уверен: наиболее достойной кандидатурой на пост нового главы российского государства мог бы стать Евгений Примаков. В этой связи Юрий Лужков еще раз подчеркнул, что не станет выдвигать св

In [56]:
title_pred(10)

Text:
Глобальное потепление приведет к высвобождению смертельноопасных для человечества древних вирусов, находящихся в "ледяныхсаркофагах" полюсов Земли. С таким апокалиптическимпредсказанием выступил сегодня журнал "New Scientist",основываясь на выводах ученых, подтвержденных недавней находкойво льдах Гренландии древнейшего вируса. Таяние льдов, как явствует из сообщения, может привести к появлению в атмосфере и мировом океане древних штаммов оспы, полиомиелита и гриппа, от которых нет "противоядия". Разносимые ветром и течениями, они могут вызвать на Земле повальные эпидемии и хаос. Группа американских ученых из Сиракузского иГосударственного университетов в Нью-Йорке, поведавших о своейнаходке британскому еженедельнику, сообщила, что возрастисследуемых льдов в Гренландии колебался от 500 лет до 140 тысячлет. Вирус, по словам ученых, надежно защищает протеиноваяоболочка, и при благоприятных условиях он вполне может "заработать".Специалисты утверждают, что даже незначительное повышени

In [57]:
title_pred(20)

Text:
Меры обеспечения безопасности правительственных и частных компьютерных сетей, принимаемые администрацией Клинтона, явно недостаточны для противостояния существующей угрозе взломов. Об этом проинформировали эксперты конгресса США, представившие вчера доклад в Конгресс, сообщает Reuters. Доклад был подготовлен по запросу Роберта Беннета, возглавляющего комитет Конгресса по "проблеме-2000", представителя республиканской партии от штата Юта. Согласно изложенным в документе данным, число попыток взлома компьютерных сетей возросло с 1334 в 1993 году до 4398 в первой половине 1999 года (что естественно, поскольку число подключенных к Интернету компьютеров и сетевых пользователей стремительно увеличивается). По мнению экспертов, распространение таких компьютерных вирусов как "Мелисса" (начало 1999 года) и хакерская атака на Минобороны США (февраль 1998) вполне отражают уязвимость существующих систем обеспечения безопасности компьютерных сетей. Еще больше экспертов тревожат частные компью

In [58]:
title_pred(30)

Text:
3 октября британская газета The Sunday Times со ссылкой на источники в российской разведке сообщила, что силы специального назначения ожидают приказа о начале операции по физическому уничтожению Шамиля Басаева и Хаттаба. Как утверждает газета, разработаны два сценария уничтожения лидеров боевиков. Согласно первому, войска Басаева и Хаттаба будут взяты в окружение группировкой федеральных сил в количестве 50,000 человек, после чего по террористам будут нанесены бомбовые удары с воздуха. По другому плану, предполагается нанести ракетный удар по Басаеву, используя для наведения сигнал, излучаемый его спутниковым телефоном. Подобным образом в 1996 году был уничтожен бывший президент Чечни Джохар Дудаев. По словам газеты, приказ должен поступить от премьер-министра РФ Владимира Путина. В минувшее воскресенье в интервью радиостанции "Эхо Москвы" бывший российский премьер Сергей Степашин высказался за физическое устранение полевых командиров Шамиля Басаева и Хаттаба. Комментируя сообщен